In [1]:
!pip install missingno

In [2]:
import pandas as pd 
import numpy as np
from collections import Counter as c 
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder 
from sklearn.linear_model import LogisticRegression  
import pickle

In [3]:
pwd

'/home/wsuser/work'

In [ ]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Tb6UaWADC_X0XMLmudtSnmSh3YuVsQN0Co1a9hct8P27',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu.cloud-object-storage.appdomain.cloud')

bucket = 'kidneydisease-donotdelete-pr-qyezpgcuq9mfps'
object_key = 'chronickidneydisease.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)
data.head()


In [5]:
data.drop(["id"],axis=1,inplace=True)

In [6]:
data.head(3)

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,48.0,80.0,1.02,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.02,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.01,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd


In [7]:
data.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu',
       'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'classification'],
      dtype='object')

In [8]:
data.columns=['age','blood_pressure','specific_gravity','albumin','sugar','red_blood_cells','pus_cell','pus_cell_clumps','bacteria','blood glucose random','blood_urea','serum_creatinine','sodium','potassium','hemoglobin','packed_cell_volume','white_blood_cell_count','red_blood_cell_count','hypertension','diabetesmellitus','coronary_artery_disease', 
'appetite','pedal_edema','anemia','class'] 
data.columns

Index(['age', 'blood_pressure', 'specific_gravity', 'albumin', 'sugar',
       'red_blood_cells', 'pus_cell', 'pus_cell_clumps', 'bacteria',
       'blood glucose random', 'blood_urea', 'serum_creatinine', 'sodium',
       'potassium', 'hemoglobin', 'packed_cell_volume',
       'white_blood_cell_count', 'red_blood_cell_count', 'hypertension',
       'diabetesmellitus', 'coronary_artery_disease', 'appetite',
       'pedal_edema', 'anemia', 'class'],
      dtype='object')

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      391 non-null    float64
 1   blood_pressure           388 non-null    float64
 2   specific_gravity         353 non-null    float64
 3   albumin                  354 non-null    float64
 4   sugar                    351 non-null    float64
 5   red_blood_cells          248 non-null    object 
 6   pus_cell                 335 non-null    object 
 7   pus_cell_clumps          396 non-null    object 
 8   bacteria                 396 non-null    object 
 9   blood glucose random     356 non-null    float64
 10  blood_urea               381 non-null    float64
 11  serum_creatinine         383 non-null    float64
 12  sodium                   313 non-null    float64
 13  potassium                312 non-null    float64
 14  hemoglobin               3

In [10]:
data['class'].unique()

array(['ckd', 'ckd\t', 'notckd'], dtype=object)

In [11]:
data['class']=data['class'].replace("ckd\t","ckd")
data['class'].unique()

array(['ckd', 'notckd'], dtype=object)

In [12]:
catcols=set(data.dtypes[data.dtypes=='O'].index.values)
print(catcols)

{'pedal_edema', 'diabetesmellitus', 'pus_cell', 'red_blood_cell_count', 'coronary_artery_disease', 'anemia', 'pus_cell_clumps', 'class', 'bacteria', 'red_blood_cells', 'appetite', 'white_blood_cell_count', 'hypertension', 'packed_cell_volume'}


In [13]:
for i in catcols:
    print("Columns :",i)
    print(c(data[i]))
    print('*'*120+'\n')

Columns : pedal_edema
Counter({'no': 323, 'yes': 76, nan: 1})
************************************************************************************************************************

Columns : diabetesmellitus
Counter({'no': 258, 'yes': 134, '\tno': 3, '\tyes': 2, nan: 2, ' yes': 1})
************************************************************************************************************************

Columns : pus_cell
Counter({'normal': 259, 'abnormal': 76, nan: 65})
************************************************************************************************************************

Columns : red_blood_cell_count
Counter({nan: 130, '5.2': 18, '4.5': 16, '4.9': 14, '4.7': 11, '3.9': 10, '4.8': 10, '4.6': 9, '3.4': 9, '3.7': 8, '5.0': 8, '6.1': 8, '5.5': 8, '5.9': 8, '3.8': 7, '5.4': 7, '5.8': 7, '5.3': 7, '4.3': 6, '4.2': 6, '5.6': 6, '4.4': 5, '3.2': 5, '4.1': 5, '6.2': 5, '5.1': 5, '6.4': 5, '5.7': 5, '6.5': 5, '3.6': 4, '6.0': 4, '6.3': 4, '4.0': 3, '4': 3, '3.5': 3, '3.3': 

In [14]:
catcols.remove('red_blood_cell_count')
catcols.remove('packed_cell_volume')
catcols.remove('white_blood_cell_count')
print(catcols)

{'pedal_edema', 'diabetesmellitus', 'pus_cell', 'coronary_artery_disease', 'anemia', 'pus_cell_clumps', 'class', 'bacteria', 'red_blood_cells', 'appetite', 'hypertension'}


In [15]:
contcols=set(data.dtypes[data.dtypes!='O'].index.values)
contcols

{'age',
 'albumin',
 'blood glucose random',
 'blood_pressure',
 'blood_urea',
 'hemoglobin',
 'potassium',
 'serum_creatinine',
 'sodium',
 'specific_gravity',
 'sugar'}

In [16]:
contcols

{'age',
 'albumin',
 'blood glucose random',
 'blood_pressure',
 'blood_urea',
 'hemoglobin',
 'potassium',
 'serum_creatinine',
 'sodium',
 'specific_gravity',
 'sugar'}

In [17]:
for i in contcols:
      print("Continous Columns :",i)
      print(c(data[i]))
      print('*'*120+'\n')

Continous Columns : hemoglobin
Counter({15.0: 16, 10.9: 8, 9.8: 7, 11.1: 7, 13.0: 7, 13.6: 7, 11.3: 6, 10.3: 6, 12.0: 6, 13.9: 6, 15.4: 5, 11.2: 5, 10.8: 5, 9.7: 5, 12.6: 5, 7.9: 5, 10.0: 5, 14.0: 5, 14.3: 5, 14.8: 5, 12.2: 4, 12.4: 4, 12.5: 4, 15.2: 4, 9.1: 4, 11.9: 4, 13.5: 4, 16.1: 4, 14.1: 4, 13.2: 4, 13.8: 4, 13.7: 4, 13.4: 4, 17.0: 4, 15.5: 4, 15.8: 4, 9.6: 3, 11.6: 3, 9.5: 3, 9.4: 3, 12.7: 3, 9.9: 3, 10.1: 3, 8.6: 3, 11.0: 3, 15.6: 3, 8.1: 3, 8.3: 3, 10.4: 3, 11.8: 3, 11.4: 3, 11.5: 3, 15.9: 3, 14.5: 3, 16.2: 3, 14.4: 3, 14.2: 3, 16.3: 3, 16.5: 3, 15.7: 3, 16.4: 3, 14.9: 3, 15.3: 3, 17.8: 3, 12.1: 2, 9.3: 2, 10.2: 2, 10.5: 2, 6.0: 2, 11.7: 2, 8.0: 2, 12.3: 2, 8.7: 2, 13.1: 2, 8.8: 2, 13.3: 2, 14.6: 2, 16.9: 2, 16.0: 2, 14.7: 2, 16.6: 2, 16.7: 2, 16.8: 2, 15.1: 2, 17.1: 2, 17.2: 2, 17.4: 2, 5.6: 1, 7.6: 1, 7.7: 1, nan: 1, nan: 1, 12.9: 1, nan: 1, nan: 1, nan: 1, nan: 1, 6.6: 1, nan: 1, nan: 1, 7.5: 1, nan: 1, nan: 1, 4.8: 1, nan: 1, nan: 1, 7.1: 1, nan: 1, nan: 1, nan: 1, 9.2: 1,

In [18]:
contcols.remove('specific_gravity')
contcols.remove('albumin')
contcols.remove('sugar')
print(contcols)

{'hemoglobin', 'age', 'blood_pressure', 'blood glucose random', 'blood_urea', 'sodium', 'potassium', 'serum_creatinine'}


In [19]:
contcols.add('red_blood_cell_count') 
contcols.add('packed_cell_volume')
contcols.add('white_blood_cell_count')
print(contcols)

{'hemoglobin', 'age', 'red_blood_cell_count', 'blood_pressure', 'blood glucose random', 'blood_urea', 'sodium', 'potassium', 'white_blood_cell_count', 'serum_creatinine', 'packed_cell_volume'}


In [20]:
catcols.add('specific_gravity')
catcols.add('albumin')
catcols.add('sugar')
print(catcols)

{'pedal_edema', 'diabetesmellitus', 'pus_cell', 'coronary_artery_disease', 'anemia', 'albumin', 'pus_cell_clumps', 'specific_gravity', 'class', 'sugar', 'bacteria', 'red_blood_cells', 'appetite', 'hypertension'}


In [21]:
data['coronary artery_disease']=data.coronary_artery_disease.replace('\tno','no')
c(data['coronary artery_disease'])

Counter({'no': 364, 'yes': 34, nan: 2})

In [22]:
data['diabetesmellitus'] = data.diabetesmellitus.replace(to_replace={'\tno':'no','\tyes':'yes',' yes':'yes'})
c(data['diabetesmellitus'])

Counter({'yes': 137, 'no': 261, nan: 2})

In [23]:
data.isnull().any()

age                         True
blood_pressure              True
specific_gravity            True
albumin                     True
sugar                       True
red_blood_cells             True
pus_cell                    True
pus_cell_clumps             True
bacteria                    True
blood glucose random        True
blood_urea                  True
serum_creatinine            True
sodium                      True
potassium                   True
hemoglobin                  True
packed_cell_volume          True
white_blood_cell_count      True
red_blood_cell_count        True
hypertension                True
diabetesmellitus            True
coronary_artery_disease     True
appetite                    True
pedal_edema                 True
anemia                      True
class                      False
coronary artery_disease     True
dtype: bool

In [24]:
data.isnull().sum()

age                          9
blood_pressure              12
specific_gravity            47
albumin                     46
sugar                       49
red_blood_cells            152
pus_cell                    65
pus_cell_clumps              4
bacteria                     4
blood glucose random        44
blood_urea                  19
serum_creatinine            17
sodium                      87
potassium                   88
hemoglobin                  52
packed_cell_volume          70
white_blood_cell_count     105
red_blood_cell_count       130
hypertension                 2
diabetesmellitus             2
coronary_artery_disease      2
appetite                     1
pedal_edema                  1
anemia                       1
class                        0
coronary artery_disease      2
dtype: int64

In [25]:
data.packed_cell_volume = pd.to_numeric(data.packed_cell_volume, errors='coerce')
data.white_blood_cell_count = pd.to_numeric(data.white_blood_cell_count, errors='coerce') 
data.red_blood_cell_count = pd.to_numeric(data.red_blood_cell_count, errors='coerce')

In [26]:
data['blood glucose random'].fillna(data['blood glucose random'].mean(), inplace=True) 
data['blood_pressure'].fillna(data['blood_pressure'].mean(), inplace=True) 
data['blood_urea'].fillna(data['blood_urea'].mean(), inplace=True) 
data['hemoglobin'].fillna(data['hemoglobin'].mean(), inplace=True) 
data['packed_cell_volume'].fillna(data['packed_cell_volume'].mean(), inplace=True) 
data['potassium'].fillna(data['potassium' ].mean(), inplace=True), 
data['red_blood_cell_count'].fillna(data['red_blood_cell_count'].mean(), inplace=True) 
data['serum_creatinine'].fillna(data['serum_creatinine'].mean(), inplace=True) 
data['sodium'].fillna(data['sodium'].mean(), inplace=True) 
data['white_blood_cell_count'].fillna(data['white_blood_cell_count'].mean(), inplace=True)

In [27]:
data['age'].fillna(data['age'].mode()[0], inplace=True) 
data['hypertension'].fillna(data['hypertension'].mode()[0], inplace=True) 
data['pus_cell_clumps'].fillna(data['pus_cell_clumps'].mode()[0], inplace=True) 
data['appetite'].fillna(data['appetite'].mode()[0], inplace=True) 
data['albumin'].fillna(data['albumin'].mode()[0], inplace=True) 
data['pus_cell'].fillna(data['pus_cell'].mode() [0], inplace=True) 
data['red_blood_cells'].fillna(data['red_blood_cells'].mode()[0], inplace=True) 
data['coronary_artery_disease'].fillna(data['coronary_artery_disease'].mode()[0], inplace=True) 
data['bacteria'].fillna(data['bacteria'].mode() [0], inplace=True) 
data['anemia'].fillna(data['anemia'].mode()[0], inplace=True) 
data['sugar'].fillna(data['sugar'].mode()[0], inplace=True) 
data['diabetesmellitus'].fillna(data['diabetesmellitus'].mode()[0], inplace=True) 
data['pedal_edema'].fillna(data['pedal_edema'].mode() [0], inplace=True) 
data['specific_gravity'].fillna(data[ 'specific_gravity'].mode()[0], inplace=True)

In [28]:
from sklearn.preprocessing import LabelEncoder
for i in catcols:
    print("LABEL ENCODING OF:",i)
    LEi=LabelEncoder()
    print(c(data[i]))
    data[i]=LEi.fit_transform(data[i])
    print(c(data[i]))
    print("*"*100)

LABEL ENCODING OF: pedal_edema
Counter({'no': 324, 'yes': 76})
Counter({0: 324, 1: 76})
****************************************************************************************************
LABEL ENCODING OF: diabetesmellitus
Counter({'no': 263, 'yes': 137})
Counter({0: 263, 1: 137})
****************************************************************************************************
LABEL ENCODING OF: pus_cell
Counter({'normal': 324, 'abnormal': 76})
Counter({1: 324, 0: 76})
****************************************************************************************************
LABEL ENCODING OF: coronary_artery_disease
Counter({'no': 364, 'yes': 34, '\tno': 2})
Counter({1: 364, 2: 34, 0: 2})
****************************************************************************************************
LABEL ENCODING OF: anemia
Counter({'no': 340, 'yes': 60})
Counter({0: 340, 1: 60})
****************************************************************************************************
LABEL ENCODING OF:

In [29]:
selcols=['red_blood_cells','pus_cell','blood glucose random','blood_urea','pedal_edema','anemia','diabetesmellitus','coronary_artery_disease']
x=pd.DataFrame(data,columns=selcols)
y=pd.DataFrame(data,columns=['class'])
print(x.shape)
print(y.shape)

(400, 8)
(400, 1)


In [30]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=2)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(320, 8)
(320, 1)
(80, 8)
(80, 1)


In [31]:
from sklearn.linear_model import LogisticRegression

In [32]:
lgr = LogisticRegression()
# x_train = x_train.values.ravel() 
# y_train = y_train.values.ravel() 
lgr.fit(x_train,y_train)

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [33]:
y_pred=lgr.predict(x_test)

In [34]:
y_pred1=lgr.predict([[129,99,1,0,0,1,0,1]])
print(y_pred)
c(y_pred)

[0 0 0 0 1 0 0 0 1 0 0 0 1 1 0 0 0 1 1 0 1 1 0 1 0 1 0 0 1 0 0 1 0 0 0 0 1
 0 0 1 0 1 0 0 0 1 0 1 1 1 0 0 0 1 0 1 0 1 1 0 0 1 1 0 0 0 0 1 0 1 1 0 0 1
 0 1 0 1 1 0]


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


Counter({0: 48, 1: 32})

In [35]:
accuracy_score(y_test,y_pred)

0.925

In [36]:
conf_mat=confusion_matrix(y_test,y_pred)
conf_mat

array([[48,  6],
       [ 0, 26]])

In [37]:
a = pickle.dump(lgr,open('CKD.pk1','wb'))

In [38]:
!ls

CKD.pk1


In [39]:
!pwd

/home/wsuser/work


In [40]:
!pip install ibm_watson_machine_learning

In [41]:
from ibm_watson_machine_learning import APIClient
wml_credentials= { 
                    "url":"https://us-south.ml.cloud.ibm.com",
                    "apikey":"e2Ss2qh2ADqYeADtcHY4kmsSqIiepUzjXpOwEXvimnKK"
}
client=APIClient(wml_credentials)


In [71]:
client

In [83]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resourses'] if item['entity']["name"]==space_name)['metadata']['id'])

In [84]:
space = client.spaces.get_details()
space

{'resources': []}

In [86]:
# space_uid = guid_from_space_name(client, 'Saru')
# print("Space UID = " + space_uid)

space_uid = 'd67e9369-4d35-4509-affd-40b7027360d5'

In [87]:
client.set.default_space(space_uid)

Cannot set Project or Space
Reason: Space with id 'd67e9369-4d35-4509-affd-40b7027360d5' does not exist


CannotSetProjectOrSpace: Cannot set Project or Space
Reason: Space with id 'd67e9369-4d35-4509-affd-40b7027360d5' does not exist